# Benchmark Vanilla GP Supervised Learning

## Preamble

This notebook provides an example of how to perform Gaussian Process Regression on a 1D manifold. In this example we consider a supervised learning scenario, namely the number of labeled data points is equivalent to the number of the sampled points from the underlying manifold.

In [1]:
import torch
import gpytorch
import numpy as np
from scipy.io import loadmat
import scipy.spatial as ss
import urllib.request
import os
from time import time
from manifold_gp.models.vanilla_gp import VanillaGP

## Dataset Preprocessing

### Load & Settings

In [2]:
torch.manual_seed(1337)
dataset = 'mnist'

data = np.loadtxt('datasets/'+dataset+'_train.csv')
sampled_x, sampled_y = data[:, 2:], data[:, 1]
rand_idx = torch.randperm(sampled_x.shape[0])
sampled_x, sampled_y = sampled_x[rand_idx], sampled_y[rand_idx]
del rand_idx

data = np.loadtxt('datasets/'+dataset+'_test.csv')
test_x, test_y = data[:, 2:], data[:, 1]
    
preprocess = False
normalize_features = False
normalize_labels = True

In [3]:
if preprocess:
    # remove coincident points
    sampled_x, id_unique = np.unique(sampled_x, axis=0, return_index=True)
    sampled_y = sampled_y[id_unique]

    # cut between 0.1 and 0.9 percentile of distances
    import faiss
    res = faiss.StandardGpuResources()
    knn = faiss.GpuIndexIVFFlat(res, sampled_x.shape[1], 1, faiss.METRIC_L2)
    knn.train(sampled_x)
    knn.add(sampled_x)
    v = np.sqrt(knn.search(sampled_x, 51)[0][:,1:])
    idx = np.argsort(v.mean(axis=1).ravel())
    percentile_start = int(np.round(idx.shape[0]*0.10))
    percentile_end = int(np.round(idx.shape[0]*0.90))
    sampled_x = sampled_x[idx[percentile_start:percentile_end], :]
    sampled_y = sampled_y[idx[percentile_start:percentile_end]]
    del knn
m = sampled_x.shape[0]

### Trainset & Testset

In [4]:
split = int(0.1 * m)

train_x, train_y = sampled_x[:split], sampled_y[:split]

train_x, train_y = torch.from_numpy(train_x).float(), torch.from_numpy(train_y).float()
test_x, test_y = torch.from_numpy(test_x).float(), torch.from_numpy(test_y).float()

if normalize_features:
    mu_x, std_x = train_x.mean(dim=-2, keepdim=True), train_x.std(dim=-2, keepdim=True) + 1e-6
    train_x.sub_(mu_x).div_(std_x)
    test_x.sub_(mu_x).div_(std_x)
    
if normalize_labels:
    mu_y, std_y = train_y.mean(), train_y.std()
    train_y.sub_(mu_y).div_(std_y)
    test_y.sub_(mu_y).div_(std_y)

### Move Data to Device

In [5]:
train_x, train_y = train_x.contiguous(), train_y.contiguous()
test_x, test_y = test_x.contiguous(), test_y.contiguous()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_x, train_y = train_x.to(device), train_y.to(device)
test_x, test_y = test_x.to(device), test_y.to(device)

## Model

In [9]:
%%capture
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_constraint=gpytorch.constraints.GreaterThan(1e-8),
    noise_prior=None  # NormalPrior(torch.tensor([0.0]).to(device),  torch.tensor([1/9]).sqrt().to(device))
)

kernel = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
# gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

model = VanillaGP(train_x, train_y, likelihood, kernel).to(device)

hypers = {
    'likelihood.noise_covar.noise': 1e-2,
    'covar_module.base_kernel.lengthscale': 1.0,
    'covar_module.outputscale': 1.0,
}
model.initialize(**hypers)

## Train

In [10]:
t0 = time()
model.vanilla_train(lr=1e-2, iter=100, verbose=True)
t1 = time()
print("Time: %.2g sec" % (t1 - t0))

Iteration: 0, Loss: 0.132, Noise Variance: 0.100, Signal Variance: 1.000, Lengthscale: 1.000
Iteration: 1, Loss: 0.105, Noise Variance: 0.100, Signal Variance: 0.997, Lengthscale: 1.006
Iteration: 2, Loss: 0.147, Noise Variance: 0.099, Signal Variance: 0.994, Lengthscale: 1.013
Iteration: 3, Loss: 0.096, Noise Variance: 0.099, Signal Variance: 0.991, Lengthscale: 1.019
Iteration: 4, Loss: 0.120, Noise Variance: 0.098, Signal Variance: 0.987, Lengthscale: 1.025
Iteration: 5, Loss: 0.100, Noise Variance: 0.098, Signal Variance: 0.984, Lengthscale: 1.032
Iteration: 6, Loss: 0.090, Noise Variance: 0.097, Signal Variance: 0.981, Lengthscale: 1.038
Iteration: 7, Loss: 0.114, Noise Variance: 0.097, Signal Variance: 0.978, Lengthscale: 1.045
Iteration: 8, Loss: 0.096, Noise Variance: 0.096, Signal Variance: 0.975, Lengthscale: 1.051
Iteration: 9, Loss: 0.080, Noise Variance: 0.096, Signal Variance: 0.972, Lengthscale: 1.058
Iteration: 10, Loss: 0.055, Noise Variance: 0.095, Signal Variance: 0.

Iteration: 93, Loss: -0.493, Noise Variance: 0.061, Signal Variance: 0.727, Lengthscale: 1.662
Iteration: 94, Loss: -0.466, Noise Variance: 0.061, Signal Variance: 0.725, Lengthscale: 1.670
Iteration: 95, Loss: -0.503, Noise Variance: 0.061, Signal Variance: 0.722, Lengthscale: 1.677
Iteration: 96, Loss: -0.517, Noise Variance: 0.060, Signal Variance: 0.720, Lengthscale: 1.685
Iteration: 97, Loss: -0.502, Noise Variance: 0.060, Signal Variance: 0.717, Lengthscale: 1.692
Iteration: 98, Loss: -0.522, Noise Variance: 0.060, Signal Variance: 0.715, Lengthscale: 1.700
Iteration: 99, Loss: -0.509, Noise Variance: 0.059, Signal Variance: 0.712, Lengthscale: 1.707
Time: 0.68 sec


## Evaluation

In [11]:
%%capture
likelihood.eval()
model.eval()

## Metrics

In [12]:
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cg_tolerance(10000):
    preds_test = likelihood(model(test_x))
        
    error = test_y - preds_test.mean
    covar = preds_test.lazy_covariance_matrix.evaluate_kernel()
    inv_quad, logdet = covar.inv_quad_logdet(inv_quad_rhs=error.unsqueeze(-1), logdet=True)
    
    rmse = (error.square().sum()/test_y.shape[0]).sqrt()
    nll = 0.5 * sum([inv_quad, logdet, error.size(-1)* np.log(2 * np.pi)])/test_y.shape[0]
    
print("RMSE: ", rmse)
print("NLL: ", nll)

RMSE:  tensor(0.0574, device='cuda:0')
NLL:  tensor(-0.9145, device='cuda:0')
